Levantamos los contenedores:

In [ ]:
!docker-compose up -d --build

In [ ]:
!python -m venv venv
!source venv/bin/activate
!pip install -r requirements.txt

Es necesario activar este environment

Conectamos la API a Mongo


In [ ]:
!python api_mongo_transfer.py

Regresa el nombre de top 10 directores con mas peliculas:

In [ ]:
from pymongo import MongoClient

# Conectar a tu instancia de MongoDB en Docker
client = MongoClient('mongodb://localhost:27017/')

# Seleccionar la base de datos y la colección
db = client['moviesdb']
credits_collection = db['credits']

# Definir la consulta
pipeline = [
    {"$unwind": "$crew"},
    {"$match": {"crew.job": "Director"}},
    {"$group": {"_id": "$crew.name", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

# Ejecutar la consulta
result = list(credits_collection.aggregate(pipeline))

# Mostrar el resultado
for director in result:
    print(f"{director['_id']}, {director['count']}")



Encontrar el título y año de las películas del género "Comedy" usando la coleccion movies (movies.json):


In [ ]:
db = client['moviesdb']
movies_collection = db['movies']

# Definir la consulta
pipeline = [
    {"$sort": {"budget": -1}},
    {"$group": {"_id": {"$year": "$release_date"}, "title": {"$first": "$title"}}},
    {"$sort": {"_id": -1}}
    
]

# Ejecutar la consulta
result = list(movies_collection.aggregate(pipeline))

# Mostrar el resultado
for movie in result:
    print(f"{movie['_id']}, {movie['title']}")



Encontrar el título y año de las películas del género "Comedy"

In [ ]:
db = client['moviesdb']
movies_collection = db['movies']

# Definir la consulta
query = {"genres": {"$elemMatch": {"name": "Comedy"}}}
projection = {"title": 1, "release_date": 1}

# Ejecutar la consulta
result = list(movies_collection.find(query, projection))

# Mostrar el resultado
for movie in result:
    print(f"{movie['title']}, {movie['release_date'].year}")

# Cerrar la conexión
client.close()

Hacemos el procesamiento de la base de datos en mongo y la cargamos a Cassandra y a Neo4j

In [ ]:
!python mongo_cassandra_transfer.py

Consulta para saber información acerca de las películas que se estrenaron en el año 2019

In [ ]:
from cassandra.cluster import Cluster
import pandas as pd


# Conectar a tu instancia de Cassandra en Docker
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Utilizar tu keyspace
session.set_keyspace('mov')

# Ejecutar la primera consulta
query1 = "SELECT title, popularity, budget, revenue FROM movies WHERE release_date >= '2019-01-01' AND release_date <= '2019-12-31' ALLOW FILTERING;"
result1 = session.execute(query1)
df1 = pd.DataFrame(list(result1), columns=result1.column_names)
df1
df1 = df1.sort_values(by=['popularity'], ascending=False)
df1.reset_index(drop=True, inplace=True)
df1[['title', 'popularity']]


Consulta para saber los géneros de las películas que se estrenaron en 1992

In [ ]:
# Conectar a tu instancia de Cassandra en Docker
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Utilizar tu keyspace
session.set_keyspace('mov')

# Ejecutar la primera consulta
query2 = "SELECT title, genres FROM movies WHERE release_date >= '1992-01-01' AND release_date <= '1992-12-31' ALLOW FILTERING;"
result2 = session.execute(query2)

print(result2)

Consulta para saber cuántas personas participaron en la producción de Goodfellas


In [ ]:
query3 = "SELECT movie_id, COUNT(*) as num_personas_cast FROM movie_cast WHERE movie_id = 769 ALLOW FILTERING;"
result3 = session.execute(query3)
print(type(row))
for row in result3:
    print(row.num_personas_cast)


In [ ]:
!python mongo_neo4j_transfer.py

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

# Conexión a la base de datos Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "neoneo4j"

driver = GraphDatabase.driver(uri, auth=(username, password))


Consulta 1: Conocer la cantidad de películas por género


In [ ]:
query_1 = """
MATCH (g:Genre)<-[:IS_GENRE_OF]-(m:Movie)
RETURN g.name, COUNT(m) as moviesCount
ORDER BY moviesCount DESC
"""


Consulta 2: Encontrar películas con conexiones a múltiples géneros


In [ ]:
query_2 = """
MATCH (m:Movie)-[:IS_GENRE_OF]->(g:Genre)
WITH m, COUNT(g) as genreCount
WHERE genreCount > 1
RETURN m.title, genreCount
"""


Consulta 3: Encontrar la película más popular de un género en particular


In [ ]:
query_3 = """
MATCH (g:Genre {name: 'Drama'})<-[:IS_GENRE_OF]-(m:Movie)
WITH m, g ORDER BY m.popularity DESC LIMIT 1
RETURN m.title, m.popularity, g.name as Genre
"""


In [ ]:
with driver.session() as session:
    print("Query 1")
    result_1 = session.run(query_1)
    df_1 = pd.DataFrame(result_1.data(), columns=result_1.keys())
    display(df_1)
    
    print("Query 2")
    result_2 = session.run(query_2)
    df_2 = pd.DataFrame(result_2.data(), columns=result_2.keys())
    display(df_2)

    print("Query 3")
    result_3 = session.run(query_3)
    df_3 = pd.DataFrame(result_3.data(), columns=result_3.keys())
    display(df_3)

# Cerrar la conexión
driver.close()